<a href="https://colab.research.google.com/github/ravisiv/APIClassification/blob/main/Email_df_Tai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



#check out MXNET




# Neural Network vs. Random Forest

In [ ]:
import tensorflow.keras as keras
import tensorflow as tf
import pandas as pd
import nltk
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords


##

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Capstone/Data
data = pd.read_csv('metadata_final_II.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/Capstone/Data


In [ ]:
data.shape

(9967, 203)

In [ ]:
data.mode().iloc[0]

Unnamed: 0              2.0
attachment_count        0.0
url1                  343.0
url2                20046.0
url3                  343.0
                     ...   
header29               49.0
header30               50.0
header31               51.0
header32               50.0
target                  0.0
Name: 0, Length: 203, dtype: float64

In [ ]:
data.isnull().sum()

Unnamed: 0             0
attachment_count       0
url1                 132
url2                 234
url3                 259
                    ... 
header29            5546
header30            5893
header31            6376
header32            6700
target                 0
Length: 203, dtype: int64

In [ ]:
data.head()

,Unnamed: 0,attachment_count,url1,url2,url3,url4,url5,url6,url7,url8,...,suffix41,suffix42,header26,header27,header28,header29,header30,header31,header32,target
0,2,0,29213.0,7580.0,29214.0,29215.0,29216.0,29214.0,21061.0,7580.0,...,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,3,0,7580.0,29236.0,29237.0,29236.0,21061.0,7580.0,29236.0,29236.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
2,4,0,5600.0,29238.0,7580.0,29239.0,29240.0,29241.0,29241.0,29239.0,...,NaN,NaN,361.0,49.0,366.0,50.0,367.0,10.0,9.0,1
3,5,0,7580.0,29245.0,29246.0,29246.0,29245.0,21061.0,7580.0,29245.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
4,6,0,5599.0,5643.0,7580.0,4450.0,7570.0,7462.0,4378.0,4450.0,...,NaN,NaN,1.0,7.0,10.0,5.0,372.0,373.0,11.0,1


In [ ]:
data.fillna(value=data.mode().iloc[0], inplace=True)

In [ ]:
data.isnull().sum()

Unnamed: 0          0
attachment_count    0
url1                0
url2                0
url3                0
                   ..
header29            0
header30            0
header31            0
header32            0
target              0
Length: 203, dtype: int64

In [ ]:
import keras

from keras.layers import Input, Dense
from keras.models import Model,load_model
from keras.layers import Input, Dense
from keras.models import Model
from keras import regularizers

from keras.preprocessing.text import Tokenizer
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.callbacks import ModelCheckpoint, TensorBoard

In [ ]:
#
from sklearn import metrics
from sklearn import svm
#from sklearn.externals import joblib
from sklearn.preprocessing import LabelEncoder

In [ ]:
# max number of features
num_max = 202

In [ ]:
import time

def train_tf_idf_model(texts):
    "train tf idf model "
    tic = time.process_time()
    
    tok = Tokenizer(num_words=num_max)
    tok.fit_on_texts(texts)
    toc = time.process_time()

    print (" -----total Computation time = " + str((toc - tic)) + " seconds")
    return tok

def prepare_model_input(tfidf_model,dataframe,mode='tfidf'):
    
    "function to prepare data input features using tfidf model"
    tic = time.process_time()
    le = LabelEncoder()
    sample_texts = list(dataframe['tokenized_text'])
    sample_texts = [' '.join(x.split()) for x in sample_texts]
    
    targets=list(dataframe['target'])
    targets = [1. if x==1 else 0. for x in targets]
    sample_target = le.fit_transform(targets)
    
    if mode=='tfidf':
        sample_texts=tfidf_model.texts_to_matrix(sample_texts,mode='tfidf')
    else:
        sample_texts=tfidf_model.texts_to_matrix(sample_texts)
    
    toc = time.process_time()
    
    print('shape of target: ', sample_target.shape)
    print('shape of data: ', sample_texts.shape)
    
    print (" -----total Computation time for preparing model data = " + str((toc - tic)) + " seconds")
    
    return sample_texts,sample_target

In [ ]:
#Define Input and Output

y = data['target']
X = data.loc[:, data.columns!='target']

In [ ]:
#Splitting Training and Testing Data

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.33)
print ('train data shape: ', X_train.shape, y_train.shape)
print ('validation data shape :' , X_val.shape, y_val.shape)

train data shape:  (6677, 202) (6677,)
validation data shape : (3290, 202) (3290,)


In [ ]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_val = sc.transform(X_val)

In [ ]:
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow.python import platform
from matplotlib import pyplot as plt

In [ ]:
# Use scikit-learn to grid search the batch size and epochs
import numpy
from sklearn.model_selection import GridSearchCV
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
# Function to create model, required for KerasClassifier
def create_model():
	# create model
  model = tf.keras.Sequential()
  model.add(tf.keras.Input(shape=(202,))) #input layer
  model.add(tf.keras.layers.Dense(500, activation='swish'))
  model.add(tf.keras.layers.Dense(50, activation = 'swish'))# dense layer
  model.add(tf.keras.layers.Dense(1, activation = 'sigmoid'))# output layer
	# Compile model
  model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
  return model
# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)
# load dataset

# split into input (X) and output (Y) variables
X = X_train
Y = y_train
# create model
model = KerasClassifier(build_fn=create_model, verbose=0)
# define the grid search parameters
batch_size = [10, 20, 40, 60, 80, 100]
epochs = [10, 50, 100]
param_grid = dict(batch_size=batch_size, epochs=epochs)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3)
grid_result = grid.fit(X, Y)
# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Best: 0.984424 using {'batch_size': 10, 'epochs': 10}
0.984424 (0.002152) with: {'batch_size': 10, 'epochs': 10}
0.982626 (0.004043) with: {'batch_size': 10, 'epochs': 50}
0.983676 (0.003119) with: {'batch_size': 10, 'epochs': 100}
0.980830 (0.002494) with: {'batch_size': 20, 'epochs': 10}
0.983076 (0.001482) with: {'batch_size': 20, 'epochs': 50}
0.979931 (0.004615) with: {'batch_size': 20, 'epochs': 100}
0.980829 (0.003078) with: {'batch_size': 40, 'epochs': 10}
0.980680 (0.000973) with: {'batch_size': 40, 'epochs': 50}
0.980530 (0.004589) with: {'batch_size': 40, 'epochs': 100}
0.980080 (0.004847) with: {'batch_size': 60, 'epochs': 10}
0.980231 (0.003002) with: {'batch_size': 60, 'epochs': 50}
0.983226 (0.003674) with: {'batch_size': 60, 'epochs': 100}
0.979182 (0.002332) with: {'batch_size': 80, 'epochs': 10}
0.981878 (0.001061) with: {'batch_size': 80, 'epochs': 50}
0.980680 (0.003003) with: {'batch_size': 80, 'epochs': 100}
0.977384 (0.004062) with: {'batch_size': 100, 'epochs': 

In [ ]:
nn_predict = grid_result.predict(X_val)
NN_classufication = classification_report(y_val, nn_predict)
print(NN_classufication)

              precision    recall  f1-score   support

           0       0.98      0.99      0.99      2303
           1       0.98      0.95      0.96       987

    accuracy                           0.98      3290
   macro avg       0.98      0.97      0.98      3290
weighted avg       0.98      0.98      0.98      3290



In [ ]:
# Model Metrics
from sklearn.model_selection import cross_validate
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import classification_report

def displayModel_metrics(best_model, grid_model, features, target, cv):
    metrics = cross_validate(best_model, features, y=target, cv=cv, 
                             scoring=['accuracy','precision','recall'], return_train_score=True) 
    
    y_predict = cross_val_predict(best_model, features, target, cv=cv)

    print('\nBest Accuracy with Grid Search            : {:.3f}'.format(grid_model.best_score_))
    print('\nTraining data Metrics')
    print('\n     The average accuraccy : {:.3f}'.format(metrics['train_accuracy'].mean()))
    print('     The average precision : {:.3f}'.format(metrics['train_precision'].mean()))
    print('     The average recall    : {:.3f}'.format(metrics['train_recall'].mean()))

    print('\nTest data Metrics')
    print('\n     The average accuracy  : {:.3f}'.format(metrics['test_accuracy'].mean()))
    print('     The average precision : {:.3f}'.format(metrics['test_precision'].mean()))
    print('     The average  recall   : {:.3f}'.format(metrics['test_recall'].mean()))
    
    matrix = classification_report(target, y_predict, labels=[1,0])
    print('\nClassification report\n')
    print(matrix)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import StratifiedShuffleSplit
import warnings
warnings.filterwarnings("ignore")

# Number of trees in random forest
n_estimators = [30, 70, 100, 150]

# Number of features to consider at every split
max_features = ['auto', 'sqrt', 'log2']

# Defining the criterion
criterion = ['gini', 'entropy']

param_rf = { 
    'n_estimators': n_estimators,
    'max_features': max_features,
    'criterion': criterion 
}

cv_shuffle = StratifiedShuffleSplit(n_splits=5)

rf = RandomForestClassifier()

rf_clf = GridSearchCV(rf, param_grid = param_rf, cv = cv_shuffle, verbose= 2)

rf_clf.fit(X_train, y_train)

rf_predicted = rf_clf.predict(X_val)

rf_report = classification_report(y_val, rf_predicted)

print(rf_clf.best_params_)

print(rf_clf.best_score_)

%time

Fitting 5 folds for each of 24 candidates, totalling 120 fits
[CV] END .criterion=gini, max_features=auto, n_estimators=30; total time=   0.3s
[CV] END .criterion=gini, max_features=auto, n_estimators=30; total time=   0.3s
[CV] END .criterion=gini, max_features=auto, n_estimators=30; total time=   0.3s
[CV] END .criterion=gini, max_features=auto, n_estimators=30; total time=   0.3s
[CV] END .criterion=gini, max_features=auto, n_estimators=30; total time=   0.3s
[CV] END .criterion=gini, max_features=auto, n_estimators=70; total time=   0.7s
[CV] END .criterion=gini, max_features=auto, n_estimators=70; total time=   0.7s
[CV] END .criterion=gini, max_features=auto, n_estimators=70; total time=   0.8s
[CV] END .criterion=gini, max_features=auto, n_estimators=70; total time=   0.7s
[CV] END .criterion=gini, max_features=auto, n_estimators=70; total time=   0.7s
[CV] END criterion=gini, max_features=auto, n_estimators=100; total time=   1.0s
[CV] END criterion=gini, max_features=auto, n_e

https://github.com/jalpa015/email-spam-detection-using-keras/blob/master/Email%20Classification%20V2.ipynb

### THE END !!! STOP!!